In [215]:
import pandas as pd
import numpy as np
import numba as nb

import time
from tqdm import tqdm

### 지역

데이터 가져오기

In [436]:
df_gj = pd.read_excel('./2. 부산.xlsx')

In [437]:
df_gj

,station_name,use_date,boarding_type,users_amount
0,다대포해수욕장,2022-01-01,B,5142
1,다대포해수욕장,2022-01-01,A,4430
2,다대포해수욕장,2022-01-02,B,3429
3,다대포해수욕장,2022-01-02,A,2992
4,다대포해수욕장,2022-01-03,B,4323
...,...,...,...,...
40539,안 평,2022-06-28,A,1903
40540,안 평,2022-06-29,B,2016
40541,안 평,2022-06-29,A,1942
40542,안 평,2022-06-30,B,2258


역이름 공백제거

In [410]:
df_gj['station_name'] = df_gj['station_name'].str.strip()

승하차 수 계산

In [322]:
# 승하차 이름 변경
idx_B = df_gj[df_gj['boarding_type'].str.contains('승차')].index
df_gj.iloc[idx_B,2] = 'B'

In [323]:
idx_A = df_gj[df_gj['boarding_type'].str.contains('하차')].index
df_gj.iloc[idx_A,2] = 'A'

인덱스 별로 sum

In [284]:
# 인덱스 기준, 로우 합계 = 그 날 이용객 수 총 합
df_gj['users_amount'] = df_gj.groupby(df_gj.index).sum().sum(axis=1)

In [326]:
# 안 들어간 값 있는지 확인
df_gj[df_gj['users_amount'].isnull()]

,station_name,use_date,boarding_type,users_amount


저장

In [440]:
df_gj.to_excel('./2. 부산.xlsx', index=False)

대구-날짜 변경

In [411]:
# str으로 타입 변경, 앞에 0 붙이기
df_gj['월'] = '0' + df_gj['월'].astype(str)
df_gj['일'] = '0' + df_gj['일'].astype(str)

In [415]:
df_gj['월'] = df_gj['월'].str.split('-').str[1]

In [418]:
df_gj['일'] = df_gj['일'].str.split('-').str[-1]

In [422]:
df_gj['use_date'] = '2022' + df_gj['월'] + df_gj['일']

In [221]:
# 안됨
for i in df_gj.index:
    df_gj.loc[:,'일'][i] = df_gj['일'][i][-2:]

날짜 포맷 변경

In [438]:
df_gj['use_date'] = df_gj['use_date'].str.split('-').str[0] + df_gj['use_date'].str.split('-').str[1] + df_gj['use_date'].str.split('-').str[2]

###### 테스트

In [191]:
# 역이름 뽑기
station_nm = []
for i in df_gj['station_name']:
    if i not in station_nm:
        station_nm.append(i)

In [197]:
station_nm

['녹동',
 '소태',
 '학동증심사',
 '남광주',
 '문화전당',
 '금남로4가',
 '금남로5가',
 '양동시장',
 '돌고개',
 '농성',
 '화정',
 '쌍촌',
 '운천',
 '상무',
 '김대중컨벤션',
 '공항',
 '송정공원',
 '광주송정',
 '도산',
 '평동',
 '김대중컨벤션센터']

In [141]:
DF = df_gj.groupby(['station_name', '구분']).sum()

In [161]:
DF.loc['화정',:]

,05_06,06_07,07_08,08_09,09_10,10_11,11_12,12_13,13_14,14_15,15_16,16_17,17_18,18_19,19_20,20_21,21_22,22_23,23_24
구분,,,,,,,,,,,,,,,,,,,
A,509,1599,3619,9362,6361,5891,6486,6285,6990,7194,8939,9530,13849,18332,8882,6699,8156,5209,2673
B,2469,6215,15072,23270,12742,9403,8383,8991,9864,9510,9275,8482,9809,8881,4331,3006,2849,2119,901


In [162]:
DF.loc['화정',:].sum(axis=1)

구분
A    136565
B    155572
dtype: int64

계산식

In [66]:
# 하차 A sum
df_A = df_gj[df_gj['구분'] == 'A']
df_A = df_A.set_index(['station_name','구분'])
df_A['sum'] = df_A.sum(axis=1)

In [102]:
df_A_sum = df_A.loc[:,:]
df_A_sum = df_A_sum.reset_index()
df_A_sum = df_A_sum[['station_name','구분','sum']]

In [68]:
# 승차 B sum
df_B = df_gj[df_gj['구분'] == 'B']
df_B = df_B.set_index(['station_name','구분'])
df_B['sum'] = df_B.sum(axis=1)

In [95]:
df_B_sum = df_B.loc[:,:]

In [98]:
df_B_sum = df_B_sum.reset_index()

In [101]:
df_B_sum = df_B_sum[['station_name','구분','sum']]

In [124]:
df_B_sum[df_B_sum['station_name']=='녹동']

,station_name,구분,sum
0,녹동,B,84
20,녹동,B,52
40,녹동,B,182
60,녹동,B,178
80,녹동,B,138
...,...,...,...
2920,녹동,B,205
2940,녹동,B,116
2960,녹동,B,91
2980,녹동,B,197


In [ ]:
df_1 = pd.read_csv('./1. 대구도시철도공사_역별일별시간별승하차인원현황_20220531(작업중).csv', encoding='cp949')

In [ ]:
df_1 = df_1[['월','일']]

In [241]:
df_1[['월','일']] = '0' + df_1[['월','일']].astype(str)

C:\Users\USER\AppData\Local\Temp\ipykernel_4532\572136169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[['월','일']] = '0' + df_1[['월','일']].astype(str)


In [246]:
df_1.__getitem__('월').__setitem__('일', '0')

C:\Users\USER\AppData\Local\Temp\ipykernel_4532\3253469041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.__getitem__('월').__setitem__('일', '0')


In [251]:
df_1.__getitem__?

Signature: df_1.__getitem__(key)
Docstring: <no docstring>
File:      c:\users\user\anaconda3\envs\oasis\lib\site-packages\pandas\core\frame.py
Type:      method


In [247]:
df_1.loc.__setitem__((slice(None), ('월', '일')), '0')

C:\Users\USER\AppData\Local\Temp\ipykernel_4532\3085084109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.loc.__setitem__((slice(None), ('월', '일')), '0')


### 법정동

데이터 불러오기

In [516]:
df = pd.read_excel('./0. 부산지하철_좌표(작업중).xlsx')

In [488]:
code = pd.read_csv('./data/법정동코드.csv')

법정동

In [517]:
df

,line_name,station_name,emd,latitude,longitude
0,1호선,다대포해수욕장,부산광역시 사하구 다대동 1552-2,35.048254,128.965792
1,1호선,다대포항,부산광역시 사하구 다대동 902,35.057667,128.971268
2,1호선,낫개,부산광역시 사하구 다대동 63-13,35.055636,128.967499
3,1호선,신장림,부산광역시 사하구 장림동 380,35.074363,128.977040
4,1호선,장림,부산광역시 사하구 장림동 185-9,35.081578,128.977733
...,...,...,...,...,...
107,4호선,석대,부산광역시 해운대구 석대동 574-14,35.218123,129.136919
108,4호선,영산대,부산광역시 해운대구 반송동 698-12,35.225749,129.146240
109,4호선,윗반송,부산광역시 해운대구 반송동 385-56,35.232459,129.153831
110,4호선,고촌,부산광역시 기장군 철마면 고촌리 518-2,35.236034,129.160516


In [518]:
# 법정동 형식으로 값 변경
df['emd'] = df['emd'].str.split(' ').str[0] + ' ' + df['emd'].str.split(' ').str[1] + ' ' + df['emd'].str.split(' ').str[2]

In [519]:
df['emd_cd'] = 0

In [520]:
# 일치하는 주소의 법정동 코드 입력
for j in range(len(df['emd'])):
    name_ = df['emd'][j]

    index = code[code['법정동명'] == name_].index
    index_list = df[df['emd'].str.contains(name_)].index

    for i in index_list:
        df['emd_cd'][i] = code['법정동코드'][index]


C:\Users\USER\AppData\Local\Temp\ipykernel_13868\1504947290.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emd_cd'][i] = code['법정동코드'][index]


In [521]:
df

,line_name,station_name,emd,latitude,longitude,emd_cd
0,1호선,다대포해수욕장,부산광역시 사하구 다대동,35.048254,128.965792,2638010600
1,1호선,다대포항,부산광역시 사하구 다대동,35.057667,128.971268,2638010600
2,1호선,낫개,부산광역시 사하구 다대동,35.055636,128.967499,2638010600
3,1호선,신장림,부산광역시 사하구 장림동,35.074363,128.977040,2638010500
4,1호선,장림,부산광역시 사하구 장림동,35.081578,128.977733,2638010500
...,...,...,...,...,...,...
107,4호선,석대,부산광역시 해운대구 석대동,35.218123,129.136919,2635010200
108,4호선,영산대,부산광역시 해운대구 반송동,35.225749,129.146240,2635010100
109,4호선,윗반송,부산광역시 해운대구 반송동,35.232459,129.153831,2635010100
110,4호선,고촌,부산광역시 기장군 철마면,35.236034,129.160516,2671033000


저장

In [522]:
df.to_excel('./3. 부산.xlsx', index=False)

### 지역별 병합, 포맷 맞추기

In [103]:
# 데이터 가져오기
df2 = pd.read_excel('./2. 부산.xlsx')
df3 = pd.read_excel('./3. 부산.xlsx')

In [104]:
# 역이름 공백제거
df2['station_name'] = df2['station_name'].str.strip()
df3['station_name'] = df3['station_name'].str.strip()

In [105]:
# 임시 병합 -> 이름 수정용
DF_ = pd.merge(df2, df3, on='station_name', how='outer')

In [122]:
# 문제있는 역명 추출 -> 역명 수정
idx = DF_[DF_['users_amount'].isnull()].index
for i in idx:
    station_idx = df3[df3['station_name'] == DF_.iloc[i,0]].index
    df3['station_name'][station_idx] = df3['station_name'][station_idx].str[0] + '  ' + df3['station_name'][station_idx].str[-1]


C:\Users\USER\AppData\Local\Temp\ipykernel_4532\2978067060.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['station_name'][station_idx] = df3['station_name'][station_idx].str[0] + '  ' + df3['station_name'][station_idx].str[-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_4532\2978067060.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['station_name'][station_idx] = df3['station_name'][station_idx].str[0] + '  ' + df3['station_name'][station_idx].str[-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_4532\2978067060.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [ ]:
# 다 바뀌었는지 확인
DF[DF['emd_cd'].isnull()]

In [ ]:
# 역이름으로 초종 병합
DF = pd.merge(df2, df3, on='station_name', how='left')

In [127]:
# 저장
DF.to_excel('./4.부산.xlsx', index=False)

### 전 지역 병합

In [153]:
# 수도권 외 지역 파일 가져오기
df_gj = pd.read_excel('./4.광주.xlsx')
df_dg = pd.read_excel('./4.대구.xlsx')
df_dj = pd.read_excel('./4.대전.xlsx')
df_bs = pd.read_excel('./4.부산.xlsx')

In [154]:
# 베이스 파일 가져오기
df = pd.read_csv('./data/local_subway_user_station_202207191126.csv')

In [155]:
# 컬럼 순서 변경
df_gj = df_gj[['boarding_type', 'emd_cd', 'latitude', 'line_name','longitude', 'station_name', 'use_date', 'users_amount']]
df_dg = df_dg[['boarding_type', 'emd_cd', 'latitude', 'line_name','longitude', 'station_name', 'use_date', 'users_amount']]
df_dj = df_dj[['boarding_type', 'emd_cd', 'latitude', 'line_name','longitude', 'station_name', 'use_date', 'users_amount']]
df_bs = df_bs[['boarding_type', 'emd_cd', 'latitude', 'line_name','longitude', 'station_name', 'use_date', 'users_amount']]

In [156]:
# 전체 데이터 병합
df_sub = pd.concat([df_gj,df_dg,df_dj,df_bs, df], axis=0)

In [160]:
# 컬럼 순서 정렬
df_sub = df_sub[['id', 'created_at', 'updated_at', 'boarding_type', 'emd_cd', 'latitude', 'line_name', 'longitude', 'station_name', 'use_date', 'users_amount']]

In [205]:
# id 설정
df_sub = df_sub.reset_index(drop=False)
df_sub = df_sub.drop(columns=['id'])
df_sub = df_sub.rename(columns={'index':'id'})

In [211]:
from datetime import datetime

In [179]:
# 생성, 수정 날짜 입력
datetime.today().strftime("%Y-%m-%d %H:%M:%S")
df_sub[['created_at','updated_at']] = datetime.today().strftime("%Y-%m-%d %H:%M:%S")

In [207]:
# csv로 저장
df_sub.to_csv('./local_subway_user_station_20220720.csv', index=False)

### 확인

In [208]:
DF = pd.read_csv('./local_subway_user_station_20220720.csv')

In [210]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116124 entries, 0 to 116123
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             116124 non-null  int64  
 1   created_at     116124 non-null  object 
 2   updated_at     116124 non-null  object 
 3   boarding_type  116124 non-null  object 
 4   emd_cd         116124 non-null  int64  
 5   latitude       116124 non-null  float64
 6   line_name      116124 non-null  object 
 7   longitude      116124 non-null  float64
 8   station_name   116124 non-null  object 
 9   use_date       116124 non-null  int64  
 10  users_amount   116124 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 9.7+ MB
